In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd "gdrive/My Drive/TopicModelling"

In [ ]:
!python -m spacy download fr_core_news_sm

In [ ]:
!pip install pyLDAvis

In [ ]:
import pandas as pd
import numpy as np
import warnings
from collections import Counter

from utilsTM import corpusVisualizer, Output, deTokenize, compareCorpuses #Output is from ipywidgets

from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases
from gensim.models.ldamodel import LdaModel, CoherenceModel
from gensim.corpora.dictionary import Dictionary

import pyLDAvis
import pyLDAvis.gensim  

import spacy

import nltk

In [ ]:
warnings.filterwarnings(action="ignore")

In [ ]:
with open(file="frenchVerbs.txt",mode="r") as f:
  FRENCHVERBS = list(f.readlines())
  FRENCHVERBS = [verb.replace("\n","").strip() for verb in FRENCHVERBS]

In [ ]:
FRENCHVERBS

In [ ]:
'''
from nltk.corpus import twitter_samples
nltk.download('twitter_samples')
tweets = twitter_samples.strings('tweets.20150430-223406.json')
'''

In [ ]:
#a correct implementation would be to have processing pipeline taking in the same things and trying them in different orders.

In [ ]:
corpus0 = pd.read_csv("CommercialRegisterData.csv").Objet.to_list()

In [ ]:
#corpus0 = pd.read_csv("GenevaCommercialRegisterData.csv").Objet.to_list()[:2000]

In [ ]:
#small UI to vizualize the corpus more easily
Out0 = Output()
corpusVisualizer(corpus0,Out0)

In [ ]:
#a lot of doc contain (cf. statuts pour but complet), (cf. but de l'établissement principal), ... at the end of the description. We remove them. 
corpus1 = [doc[:doc.find("(cf")] for doc in corpus0]

In [ ]:
Out1 = Output()
corpusVisualizer(corpus1,Out1)

In [ ]:
# VERB detection is so shit with spacy that we had to scrape a list of all french verbs
v = set([word for doc in corpus2 for word in doc if word in FRENCHVERBS])
v

In [ ]:
len(v)

442

In [ ]:
#2. Perform a number of preliminary preprocessing steps using gensim's simple_preprocess() (remove punctations, lowercase etc...).
corpus2 = [simple_preprocess(doc,deacc=False,max_len=30) for doc in corpus1]

In [ ]:
Out2 = Output()
corpusVisualizer(deTokenize(corpus2),Out2)

In [ ]:
#3. Remove stopwords, lemmatize
nlp = spacy.load("fr_core_news_sm")
corpus3 = []
for doc2 in deTokenize(corpus2):
  doc2 = nlp(doc2)
  doc3 = [token.lemma_ for token in doc2 if not token.is_stop if not token.text in FRENCHVERBS]
  corpus3.append(doc3)

In [ ]:
#2. Remove accents
corpus4 = [simple_preprocess(doc,deacc=True,max_len=30) for doc in deTokenize(corpus3)]

In [ ]:
Out3 = Output()
corpusVisualizer(deTokenize(corpus3),Out3)

In [ ]:
#stemming, we try the french stemmer from nltk
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()
corpus4 = [[stemmer.stem(word) for word in doc] for doc in corpus3]

In [ ]:
Out4 = Output()
corpusVisualizer(deTokenize(corpus4),Out4)

In [ ]:
#4. detect bigrams
corpus5 = list(Phrases(sentences=corpus3,min_count=2)[corpus4])

In [ ]:
# detect trigrams
#corpus4 = list(Phrases(sentences=corpus4,min_count=2)[corpus4])

In [ ]:
Out4 = Output()
corpusVisualizer(deTokenize(corpus4),Out4)

In [ ]:
#5. Remove pervasive words, which are present in most of the documents and are noise from a domain specific vocabulary (business) rather than discriminative features
#a. Take the most frequent word, manually set the threshold for the most common word number and keep some words which might be due to a frequent topic
cnt = Counter([word for doc in corpus5 for word in doc])
keep = ["financier","immobilier","courtage","batiment","materiel","importation_exportation","informatique","bien_immobilier","investissement","industriel","public"]
remove = [k for k,_ in cnt.most_common(63) if k not in keep]
manual = ["lfaie","but","pays","genre","aupres","groupe","lien","principalement","commercer","rapport","edition","geneve","premier","tiers"] #remove adverbes...
remove = remove + manual
#b. Automatically detect topics that are domain specific and not disciminative, by looking at the entropy of topic distributions

#remove words
corpus6 = [[word for word in doc if word not in remove] for doc in corpus5]

In [ ]:
 #To compare corpuses
Out = Output()
compareCorpuses(corpus0,corpus4,Out)

In [ ]:
#Final corpus remove empty docs
corpusFinal = [doc for doc in corpus6 if doc]

In [ ]:
id2word = Dictionary(documents=corpusFinal)

In [ ]:
corpusBow = [id2word.doc2bow(doc) for doc in corpusFinal]

In [ ]:
ntopics = 15
rs = 100
model = LdaModel(corpus=corpusBow,num_topics=ntopics,id2word=id2word,chunksize=2000,
                 passes=20,minimum_probability=0.0001,random_state=rs,per_word_topics=False)
coherence = CoherenceModel(model=model,texts=corpusFinal,dictionary=id2word).get_coherence()
print("Average correlation between multi-words in topics",round(coherence,4))

Average correlation between multi-words in topics 0.5005


In [ ]:
#80 not so bad,10 not bad, 15 0.5 !!! #sometimes bigrams are confusing the thign
#model.save(fname="model80topics")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpusBow, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
1     -0.052461  0.002938       1        1  7.134648
2     -0.325632 -0.060223       2        1  6.645026
0     -0.230944  0.043142       3        1  6.403727
17    -0.000338  0.176145       4        1  5.406419
14    -0.004760 -0.152142       5        1  5.142902
11     0.051779 -0.276054       6        1  4.981830
23    -0.223327  0.079722       7        1  4.892581
22     0.079084  0.142357       8        1  4.638106
19     0.074870 -0.137153       9        1  3.979075
24     0.082277  0.111151      10        1  3.738309
3      0.001029  0.072698      11        1  3.703949
18    -0.210903 -0.034366      12        1  3.654801
6      0.078415  0.098213      13        1  3.613447
13     0.091853 -0.168797      14        1  3.599911
8      0.035917  0.045325      15        1  3.551173
9      0.061981  0.064690      16        1  3.436652
20     0.080956  0.042275      17        1  3.263631
5      0.035348 -0.008131      18        1  3.155599
15     0.082449 -0.000741      19        1  2.992185
16     0.050129 -0.064120      20        1  2.987382
21     0.059263  0.091610      21        1  2.802638
7      0.027050 -0.102294      22        1  2.797961
12     0.060753 -0.007810      23        1  2.586241
10     0.089354  0.052190      24        1  2.512269
4      0.005859 -0.010626      25        1  2.379539, topic_info=                     Term         Freq        Total Category  logprob  loglift
11             immobilier  1174.000000  1174.000000  Default  30.0000  30.0000
142             financier   971.000000   971.000000  Default  29.0000  29.0000
218          informatique   525.000000   525.000000  Default  28.0000  28.0000
6                courtage   482.000000   482.000000  Default  27.0000  27.0000
134              batiment   403.000000   403.000000  Default  26.0000  26.0000
...                   ...          ...          ...      ...      ...      ...
578               holding    31.933619    40.191482  Topic25  -4.1530   3.5083
549           electricite    33.205306    70.152070  Topic25  -4.1139   2.9903
1341              maniere    21.709801    64.596887  Topic25  -4.5389   2.6479
531   prise_participation    26.668932   283.612825  Topic25  -4.3331   1.3742
916               pouvoir    19.088626    90.160972  Topic25  -4.6675   2.1858

[1279 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
2841     24  0.954532       academique
966       6  0.805943       accessoire
966      14  0.190653       accessoire
3187     13  0.975886  accessoire_mode
253       4  0.851606   accompagnement
...     ...       ...              ...
6844     24  0.946669     voiture_moto
564      22  0.911496              vol
699      13  0.302618           voyage
699      22  0.680891           voyage
400       1  0.986006              web

[2024 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 18, 15, 12, 24, 23, 20, 25, 4, 19, 7, 14, 9, 10, 21, 6, 16, 17, 22, 8, 13, 11, 5])

In [ ]:
from gensim import similarities
index = similarities.MatrixSimilarity(model[corpusBow])

In [ ]:
sims = index[vec_lsi]

In [ ]:
vec_lsi

[(0, 0.02222222),
 (1, 0.02222222),
 (2, 0.02222222),
 (3, 0.02222222),
 (4, 0.02222222),
 (5, 0.02222225),
 (6, 0.02222222),
 (7, 0.02222222),
 (8, 0.02222222),
 (9, 0.02222222),
 (10, 0.02222222),
 (11, 0.02222222),
 (12, 0.02222222),
 (13, 0.3555555),
 (14, 0.3555556)]

In [ ]:
sorted(enumerate(sims), key=lambda item: -item[1])

In [ ]:
doc = "horlogerie metal precieux"
vec_bow = id2word.doc2bow(doc.lower().split())
vec_lsi = model[vec_bow]  # convert the query to LSI space
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in sims[:200]:
    print(i,s, corpus0[i])